# Calculate the intrinsec value of a company

## Formula to use


Owner's Earnings = Operating Cash Flow* - Maintenance Capital Expenditure

*\*Cash Flow From Operating Activities*


**Operating Cash Flow** = *A measure of the amount of cash generated by a company's normal business operations*

**Maintenance Capital Expenditure** = *CapEx that is necesary for the company to continue operating in it's current form*

This is not easy to come by in the statements as it's not mandatory to report.

Can be replaced by *Total Capital Expenditure*, which is *CapEx that is necesary for the company to continue operating in it's current form, and grow the business*

If this is the case, the formula changes to:

Free Cash Flow = Operating Cash Flow - Total Capital Expenditure*

*\*Total Capital Expenditure - Payments for aquisition of property , plant and equipment*

You can get the Free Cash Flow directly on the statements.



## Example

We are going to buy the corner shop.

This is how the Free Cash Flow looks year by year:

| Year | Free Cash Flow | Growth |
|------|----------------|--------|
|2017  |$9,010          |N/A     |
|2018  |$10,000         |11%     |
|2019  |$12,000         |20%     |
|2020  |$16,000         |33%     |
|2021  |$21,000         |31%     |

Calculate the business value into the future:

First calculate an average growth rate year by year:

In [39]:
def format_amount(value: float)-> str:
	return "${:,}".format(value)

def calculate_growth_rate(initial: float, last: float)-> float:
	diff = (last - initial) / initial * 100
	return round(diff)

In [40]:
import pandas as pd

free_cash_flows = [9010, 10000, 12000, 16000, 21000]
start_year = 2017

data = {"Year": [], "Free Cash Flow": [], "Growth": []}

total = 0
for i, fcf in enumerate(free_cash_flows):
	if i == 0:
		continue

	prev = free_cash_flows[i - 1]
	growth = calculate_growth_rate(prev, fcf)

	data["Year"].append(start_year + i)
	data["Free Cash Flow"].append(format_amount(fcf))
	data["Growth"].append(f"{growth}%")

	total += fcf

df = pd.DataFrame(data)
print(df)

print(f"Total current value is {format_amount(total)}")

   Year Free Cash Flow Growth
0  2018        $10,000    11%
1  2019        $12,000    20%
2  2020        $16,000    33%
3  2021        $21,000    31%
Total current value is $59,000


## Calculate the undiscounted value of the company

- Calculate the average growth rate

- Apply the growth rate for 10 years into the future

- Sum up all the cash flows per year

In [41]:
import pandas as pd


def calculate_growth(base_value: float, growth_rate: float)-> float:
	return base_value + (base_value * growth_rate)

def calculate_avr_growth_rate(values: list)-> float:
	values = [v for v in values if v != 0]
	total = 0
	for v in values:
		total += v
	return total / len(values)

last_free_cash_flow = 21000
avr_growth_rate = calculate_avr_growth_rate([0.11, 0.20, 0.33, 0.31])

print(f"Average Growth Rate is {avr_growth_rate}")

start_year = 2022
nr_of_years = 10

data = {
	"Year": [], 
	"Free Cash Flow": [], 
	"FCF Raw": [], 
	"Growth": [],
}
total = 0

for i in range(nr_of_years):
	cash_flow = calculate_growth(last_free_cash_flow, avr_growth_rate)
	cash_flow = round(cash_flow, 2)

	total += cash_flow
	last_free_cash_flow = cash_flow

	data["Year"].append(start_year + i)
	data["Free Cash Flow"].append(format_amount(cash_flow))
	data["FCF Raw"].append(cash_flow) # Add this to be processed later
	data["Growth"].append(f"{avr_growth_rate * 100}%")

df = pd.DataFrame(data)
print(df[["Year", "Free Cash Flow", "Growth"]])

print(f"Total value after {nr_of_years} years is {format_amount(total)}")

Average Growth Rate is 0.2375
   Year Free Cash Flow  Growth
0  2022      $25,987.5  23.75%
1  2023     $32,159.53  23.75%
2  2024     $39,797.42  23.75%
3  2025     $49,249.31  23.75%
4  2026     $60,946.02  23.75%
5  2027      $75,420.7  23.75%
6  2028     $93,333.12  23.75%
7  2029    $115,499.74  23.75%
8  2030    $142,930.93  23.75%
9  2031    $176,877.03  23.75%
Total value after 10 years is $812,201.3


## Add the sell price of the business

Now that we got the total value of the cash flow for 10 years, we need to assume that we will also sell the company for a price.

Let's take the sell price as being 10x the future free cashflow.

In [42]:
sell_price = last_free_cash_flow * 10

print(f"Selling price for the company is {format_amount(sell_price)}")

total_value = total + sell_price

print(f"Selling price + free cash flow we made over the {nr_of_years} we get {format_amount(total_value)}")

Selling price for the company is $1,768,770.3
Selling price + free cash flow we made over the 10 we get $2,580,971.6


## Money received today is worth more than money received in the future

We need to discount each year free cash flow by 15% to account fo inflation and interest lost.

At the end we will also discount the sell price of the company and sum everything up.

In [43]:
discount_rate = 0.15

def discount_amount(value: float)-> float:
	return value - (discount_rate * value)

column_name = "Discounted FCF"
raw_column_name = "DFCF raw"

discounted_total = 0
discounted = []
for fcf in df["FCF Raw"]:
	discounted_fcf = round(discount_amount(fcf), 2)
	discounted.append(discounted_fcf)
	discounted_total += discounted_fcf

df.loc[:, column_name] = [ format_amount(d) for d in discounted ]
df.loc[:, raw_column_name] = discounted
print(df[["Year", "Free Cash Flow", "Growth", column_name]])

total_discounted_company_value = discount_amount(sell_price) + discounted_total
print(f"Discounted Company Value Price is {format_amount(total_discounted_company_value)}")

   Year Free Cash Flow  Growth Discounted FCF
0  2022      $25,987.5  23.75%     $22,089.38
1  2023     $32,159.53  23.75%      $27,335.6
2  2024     $39,797.42  23.75%     $33,827.81
3  2025     $49,249.31  23.75%     $41,861.91
4  2026     $60,946.02  23.75%     $51,804.12
5  2027      $75,420.7  23.75%      $64,107.6
6  2028     $93,333.12  23.75%     $79,333.15
7  2029    $115,499.74  23.75%     $98,174.78
8  2030    $142,930.93  23.75%    $121,491.29
9  2031    $176,877.03  23.75%    $150,345.48
Discounted Company Value Price is $2,193,825.875


## Apply the formula for a real company

We are going to pick Apple and calculate it's intrinsec value.

Formula that we are going to apply simplified.

DCF = CF1 / (1 + R)\*\*1 + CF2 / (1 + R)\*\*2 + ... + CFn / (1 + R)\*\*n

Where:

- CFi = Cash flow for a period of time (preferably per year)

- R = interest rate

- n = period of time

In [52]:
from yahoo_fin import stock_info as stock
from pprint import pprint


discount_rate = 0.15 # or interest rate
current_cash_flow = 0
years = 5

cash_flows = stock.get_cash_flow("aapl", True)
free_cashflows = []

data = {
	"Year": [], 
	"Free Cash Flow": [],
	"Free Cash Flow Raw": [],
	"Growth": [], 
	"Growth Raw": [],
}

start_year = None
for t, cf in sorted(cash_flows.to_dict().items()):
	if start_year is None:
		start_year = t.year

	free_cashflow = cf["totalCashFromOperatingActivities"] - cf["capitalExpenditures"]
	free_cashflows.append(free_cashflow)

for i, fcf in enumerate(free_cashflows):

	data["Year"].append(start_year + i)
	data["Free Cash Flow"].append(format_amount(fcf))
	data["Free Cash Flow Raw"].append(fcf)

	if i == 0:
		data["Growth"].append("N/A")
		data["Growth Raw"].append(0)

		continue

	prev = free_cashflows[i - 1]
	growth = calculate_growth_rate(prev, fcf)

	data["Growth"].append(f"{growth}%")
	data["Growth Raw"].append(growth / 100)

df = pd.DataFrame(data)
print(df[["Year", "Free Cash Flow", "Growth"]])

avr_growth_rate = round(calculate_avr_growth_rate(df["Growth Raw"]), 2)
print(avr_growth_rate)

def calculate_dcf(current_cf: float, growth_rate: float, years: int, discount_rate: float)-> float:
	dcf = 0
	for i in range(1, years + 1, 1):
		dcf += current_cf + (current_cf * growth_rate * i) / (1 + discount_rate)**i

	return dcf

last_cf = df["Free Cash Flow Raw"].iloc[-1]
dcf = calculate_dcf(last_cf, avr_growth_rate, 5, 0.15)
print(f"Discounted CF ${dcf}")

   Year    Free Cash Flow Growth
0  2018   $90,747,000,000    N/A
1  2019   $79,886,000,000   -12%
2  2020   $87,983,000,000    10%
3  2021  $115,123,000,000    31%
0.1
Discounted CF $680691191707.931
